In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from gensim.models import Word2Vec
import multiprocessing

In [13]:
### 计算是2018年的第几天
def is_leap_year(year):
    if year % 4 == 0 and year % 100 != 0 or year % 400 == 0:
        return True
    return False
def which_day(df):
    year = int(df.split('/')[0])
    month = int(df.split('/')[1])
    day = int(df.split('/')[2])
    month_of_day31 = [1,3,5,7,8,10,12]
    month_of_day30 = [4,6,9,11]
    feb_month = 2
    if month ==1 :
        return day
    if month == 2:
        return day + 31
    days_of_31_num = 0
    days_of_30_num = 0
    ## 31天月份数
    for days_of_31 in month_of_day31:
        if days_of_31 < month:
            days_of_31_num += 1
        else:
            break
    ## 30天月份数
    for days_of_30 in month_of_day31:
        if days_of_30 < month:
            days_of_30_num += 1
        else:
            break
    return days_of_31_num * 31 + days_of_30_num * 30 +(29 if is_leap_year(year) else 28)

In [25]:
def parseData(data,is_test=True):       
    # buildYear 暂无信息 处理(众数填充)
    tmp = data['buildYear'].copy()
    tmp2 = tmp[tmp!='暂无信息'].astype('int')
    tmp[tmp=='暂无信息'] = tmp2.mode()[0]
    data['buildYear'] = tmp
    data['buildYear'] = data['buildYear'].astype('int')
    
    # 缺失值处理
    data['pv'].replace(to_replace=np.nan,value=data['pv'].mean(), inplace=True)
    data['uv'].replace(to_replace=np.nan,value=data['uv'].mean(), inplace=True)

 
    # 拆分 houseType
    data[['bedroom', 'hall', 'wc', 'null']] = data['houseType'].str.split('[室厅卫]', expand=True)
    data[['bedroom', 'hall', 'wc']] = data[['bedroom', 'hall', 'wc']].astype('int64')
    data.drop(['null','houseType'], axis=1, inplace=True)
    # rentType未知方式 处理
    data.loc[(data['rentType']=='--','rentType')] = '未知方式'

    
    # 拆分 tradeTime 
    data['tradeMonth'] = data['tradeTime'].apply(lambda x:x.split('/')[1]).astype('int64')
    data['day'] = data['tradeTime'].apply(lambda x: int(x.split('/')[1])).astype('int64') ##将时间特征进行放大  
    ### 看看交易时间是2018年的第几天
    data['whichday'] = data['tradeTime'].apply(which_day)
    data['whichday'] = data['whichday'].astype('int64')
    data = data.drop(['tradeTime'],axis=1)
    if is_test:
        # 去除ID列
        data.drop(['ID'], axis=1, inplace=True)
    # 去除部分特征
    data.drop(['city'], axis=1, inplace=True)
    return data

In [26]:
# Clean Data
def cleanData(data):    
    data.drop(data[(data['tradeMoney']>16000)].index,inplace=True) 
    data.drop(data[(data['area']>160)].index,inplace=True)
    data.drop(data[(data['tradeMoney']<800)].index,inplace=True)
    data.drop(data[(data['totalFloor']==0)].index,inplace=True)

    # 深度清理
    data.drop(data[(data['region']=='RG00001')&(data['tradeMoney']<1000)&(data['area']>50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['tradeMoney']>25000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['area']>250)&(data['tradeMoney']<20000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['area']>400)&(data['tradeMoney']>50000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['area']>100)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00002') & (data['area']<100)&(data['tradeMoney']>60000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['area']<300)&(data['tradeMoney']>30000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003')&(data['tradeMoney']<500)&(data['area']<50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003')&(data['tradeMoney']<1500)&(data['area']>100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003')&(data['tradeMoney']<2000)&(data['area']>300)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003')&(data['tradeMoney']>5000)&(data['area']<20)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['area']>600)&(data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00004')&(data['tradeMoney']<1000)&(data['area']>80)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['tradeMoney']<200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005')&(data['tradeMoney']<2000)&(data['area']>180)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005')&(data['tradeMoney']>50000)&(data['area']<200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['area']>200)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00007') & (data['area']>100)&(data['tradeMoney']<2500)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['area']>200)&(data['tradeMoney']>25000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['area']>400)&(data['tradeMoney']<15000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010')&(data['tradeMoney']<3000)&(data['area']>200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010')&(data['tradeMoney']>7000)&(data['area']<75)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010')&(data['tradeMoney']>12500)&(data['area']<100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00004') & (data['area']>400)&(data['tradeMoney']>20000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&(data['tradeMoney']<2000)&(data['area']>80)].index,inplace=True)
    data.drop(data[(data['region']=='RG00009') & (data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00009') & (data['area']>300)].index,inplace=True)
    data.drop(data[(data['region']=='RG00009')&(data['area']>100)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00011')&(data['tradeMoney']<10000)&(data['area']>390)].index,inplace=True)
    data.drop(data[(data['region']=='RG00012') & (data['area']>120)&(data['tradeMoney']<5000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00013') & (data['area']<100)&(data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00013') & (data['area']>400)&(data['tradeMoney']>50000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00013')&(data['area']>80)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['area']>300)&(data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014')&(data['tradeMoney']<1300)&(data['area']>80)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014')&(data['tradeMoney']<8000)&(data['area']>200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014')&(data['tradeMoney']<1000)&(data['area']>20)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014')&(data['tradeMoney']>25000)&(data['area']>200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014')&(data['tradeMoney']<20000)&(data['area']>250)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005')&(data['tradeMoney']>30000)&(data['area']<100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005')&(data['tradeMoney']<50000)&(data['area']>600)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005')&(data['tradeMoney']>50000)&(data['area']>350)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006')&(data['tradeMoney']>4000)&(data['area']<100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006')&(data['tradeMoney']<600)&(data['area']>100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006')&(data['area']>165)].index,inplace=True)
    data.drop(data[(data['region']=='RG00012')&(data['tradeMoney']<800)&(data['area']<30)].index,inplace=True)
    data.drop(data[(data['region']=='RG00007')&(data['tradeMoney']<1100)&(data['area']>50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00004')&(data['tradeMoney']>8000)&(data['area']<80)].index,inplace=True)
    data.loc[(data['region']=='RG00002')&(data['area']>50)&(data['rentType']=='合租'),'rentType']='整租'
    data.loc[(data['region']=='RG00014')&(data['rentType']=='合租')&(data['area']>60),'rentType']='整租'
    data.drop(data[(data['region']=='RG00008')&(data['tradeMoney']>15000)&(data['area']<110)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&(data['tradeMoney']>20000)&(data['area']>110)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&(data['tradeMoney']<1500)&(data['area']<50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&(data['rentType']=='合租')&(data['area']>50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00015') ].index,inplace=True)
    data.drop(data[(data['region']=='RG00014')&(data['tradeMoney']>13000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005')&(data['tradeMoney']<1200)&(data['area']>80)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005')&(data['tradeMoney']>13000)&(data['area']<90)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005')&(data['tradeMoney']<1600)&(data['area']>100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00002')&(data['tradeMoney']>13000)&(data['area']<100)].index,inplace=True)
    data.reset_index(drop=True, inplace=True)
    
    return data

In [27]:
# Make Features
def MakeFeatures_1(data):
    # 合并车站
    data['bus_sub_num'] = data['subwayStationNum']+data['busStationNum']
    # 合并学校
    data['school_num'] = data['interSchoolNum']+data['schoolNum']+data['privateSchoolNum']
    # 合并医疗
    data['help_sum'] = data['hospitalNum']+data['drugStoreNum']
    # 合并生活设施
    data['play_sum'] = data['gymNum']+data['parkNum']+data['bankNum']
    # 合并购物
    data['shop_num'] = data['shopNum']+data['mallNum']+data['superMarketNum']
    # 其他合并
    data['totalNewTradeMoney_Workers'] = data['totalNewTradeMoney'] + data['totalWorkers']
    data['bankNum_Workers'] = data['bankNum'] + data['totalWorkers']
    data['gym_bankNum'] = data['bankNum'] + data['gymNum']
    # "板块二手房价"
    data['area_mean_price'] = (data['area']*data['tradeMeanPrice'])/1000
    # "板块新房房价"
    data['New_area_mean_price'] = (data['area']*data['tradeNewMeanPrice'])/1000
    # "板块房价"
#     data['Mean_price'] = (data['tradeMeanPrice']+data['tradeNewMeanPrice'])/2 #变差
#     data['Mean_area_mean_price'] = (data['area']*data['Mean_price'])/1000 #变差
############### Hero 特征 ###################################################################
    def get_train_mode(train, data, by, fea):
        def Encode_houseFloor(x):
            if x =='低':
                return 1
            if x =='中':
                return 2
            if x =='高':
                return 3
        if fea=='area':
            train[fea] = train[fea].apply(lambda x: round(x,-1))
        if fea=='tradeMoney':
            train[fea] = train[fea].apply(lambda x: round(x,-2)) 
        if fea=='houseFloor':
            train[fea] = train[fea].apply(Encode_houseFloor)
        gp = train.groupby(by)[fea].agg(lambda x: np.mean(pd.Series.mode(x))).reset_index()     
        gp = gp.rename(columns={fea:f'trn_{"_".join(by)}_{fea}_mode'})
        data = pd.merge(data, gp, how='left', on=by)
        return data 
    def get_train_mean(train, data, by, fea):
        gp = train.groupby(by)[fea].mean().reset_index()
        gp = gp.rename(columns={fea:f'trn_{"_".join(by)}_{fea}_mean'})
        data = pd.merge(data, gp, how='left', on=by)
        return data          
    def get_train_median(train, data, by, fea):
        gp = train.groupby(by)[fea].median().reset_index()
        gp = gp.rename(columns={fea:f'trn_{"_".join(by)}_{fea}_median'})
        data = pd.merge(data, gp, how='left', on=by)
        return data 
    def get_train_std(train, data, by, fea):
        gp = train.groupby(by)[fea].std().reset_index()
        gp = gp.rename(columns={fea:f'trn_{"_".join(by)}_{fea}_std'})
        data = pd.merge(data, gp, how='left', on=by)
        data[f'trn_{"_".join(by)}_{fea}_std'] = data[f'trn_{"_".join(by)}_{fea}_std'].fillna(0)
        return data
    def get_train_max(train, data, by, fea):
        gp = train.groupby(by)[fea].max().reset_index()
        gp = gp.rename(columns={fea:f'trn_{"_".join(by)}_{fea}_max'})
        data = pd.merge(data, gp, how='left', on=by)
        return data
    def get_train_min(train, data, by, fea):
        gp = train.groupby(by)[fea].min().reset_index()
        gp = gp.rename(columns={fea:f'trn_{"_".join(by)}_{fea}_min'})
        data = pd.merge(data, gp, how='left', on=by)
        return data
    def groupby(data,by_list): 
        # 读取全量数据                        
        train = pd.read_csv('./train_data.csv')
        train = parseData(train)
        # 数据处理
        train.loc[(train['area']>1000),'area'] = 1000
        train['area_mean_price'] = (train['area']*train['tradeMeanPrice'])/1000  
        train['New_area_mean_price'] = (train['area']*train['tradeNewMeanPrice'])/1000
#         # "板块房价"
#         train['Mean_price'] = (train['tradeMeanPrice']+train['tradeNewMeanPrice'])/2
#         train['Mean_area_mean_price'] = (train['area']*train['Mean_price'])/1000                        
#         train['pv_uv_ratio'] = train['pv']/(train['uv']+1) 
        # 对 area_mean_price统计                       
#         data = get_train_mode(train, data, by, 'area_mean_price')
        data = get_train_mean(train, data, by, 'area_mean_price')
        data = get_train_std(train, data, by, 'area_mean_price')
#         data = get_train_median(train, data, by, 'area_mean_price')
#         # 对 New_area_mean_price统计
        data = get_train_mean(train, data, by, 'New_area_mean_price')
        data = get_train_std(train, data, by, 'New_area_mean_price')
#         data = get_train_mean(train, data, by, 'Mean_price')
#         data = get_train_std(train, data, by, 'Mean_price')                        
#         data = get_train_mean(train, data, by, 'Mean_area_mean_price')
#         data = get_train_std(train, data, by, 'Mean_area_mean_price')                        
                                
#         # 对pv_uv_ratio统计
#         data = get_train_mode(train, data, by, 'pv_uv_ratio')
#         data = get_train_mean(train, data, by, 'pv_uv_ratio')
#         data = get_train_std(train, data, by, 'pv_uv_ratio')
#         data = get_train_median(train, data, by, 'pv_uv_ratio')                        
#         # 对面积做统计特征，（有用）
        data = get_train_mode(train, data, by, 'area')
#         data = get_train_mean(train, data, by, 'area')
        data = get_train_std(train, data, by, 'area')
        data = get_train_median(train, data, by, 'area')                                                       
#     #     data = get_train_max(train, data, by, 'area') #加了过拟合
#     #     data = get_train_min(train, data, by, 'area') #加了过拟合
#         # 众数                            
#         data = get_train_mode(train, data, by, 'tradeMonth') #加了变差
#         data = get_train_mode(train, data, by, 'houseFloor') #加了变差                           
#         data = get_train_mode(train, data, by, 'buildYear') #加了变差
#         data = get_train_mode(train, data, by, 'totalFloor')
        data = get_train_mode(train, data, by, 'pv')
#         data = get_train_mode(train, data, by, 'uv')
        data = get_train_mode(train, data, by, 'tradeMeanPrice')
        data = get_train_mode(train, data, by, 'tradeNewMeanPrice')                               
#         data = get_train_mode(train, data, by, 'totalTradeMoney')
        data = get_train_mode(train, data, by, 'totalNewTradeMoney')
        data = get_train_mode(train, data, by, 'saleSecHouseNum')
        data = get_train_mode(train, data, by, 'totalTradeArea')
#         data = get_train_mode(train, data, by, 'tradeSecNum')
#         data = get_train_mode(train, data, by, 'totalNewTradeArea') 
#         data = get_train_mode(train, data, by, 'tradeNewNum')
        data = get_train_mode(train, data, by, 'remainNewNum')
        data = get_train_mode(train, data, by, 'supplyNewNum')                           
        data = get_train_mode(train, data, by, 'newWorkers')  
        data = get_train_mode(train, data, by, 'bedroom')
#         data = get_train_mode(train, data, by, 'totalWorkers')                            
#         # 均值                            
#         data = get_train_mean(train, data, by, 'tradeMonth') #加了变差
#         data = get_train_mean(train, data, by, 'houseFloor') #加了变差                           
#         data = get_train_mean(train, data, by, 'buildYear') #加了变差
#         data = get_train_mean(train, data, by, 'totalFloor')
#         data = get_train_mean(train, data, by, 'pv')
#         data = get_train_mean(train, data, by, 'uv')
#         data = get_train_mean(train, data, by, 'tradeMeanPrice')
#         data = get_train_mean(train, data, by, 'tradeNewMeanPrice')                            
#         data = get_train_mean(train, data, by, 'totalTradeMoney')
#         data = get_train_mean(train, data, by, 'totalNewTradeMoney')
        data = get_train_mean(train, data, by, 'saleSecHouseNum')
#         data = get_train_mean(train, data, by, 'totalTradeArea')
#         data = get_train_mean(train, data, by, 'tradeSecNum')
#         data = get_train_mean(train, data, by, 'totalNewTradeArea') 
#         data = get_train_mean(train, data, by, 'tradeNewNum')
#         data = get_train_mean(train, data, by, 'remainNewNum')
        data = get_train_mean(train, data, by, 'supplyNewNum')
        data = get_train_mean(train, data, by, 'newWorkers')
#         data = get_train_mean(train, data, by, 'bedroom')
#         data = get_train_mean(train, data, by, 'totalWorkers') # 特征选择删除了    
#         # 方差
#         data = get_train_std(train, data, by, 'tradeMonth') #加了变差
#         data = get_train_std(train, data, by, 'houseFloor') #加了变差                           
#         data = get_train_std(train, data, by, 'buildYear') #加了变差                            
        data = get_train_std(train, data, by, 'totalFloor')
        data = get_train_std(train, data, by, 'pv')
#         data = get_train_std(train, data, by, 'uv')
        data = get_train_std(train, data, by, 'tradeMeanPrice')
        data = get_train_std(train, data, by, 'tradeNewMeanPrice')                               
        data = get_train_std(train, data, by, 'totalTradeMoney')
        data = get_train_std(train, data, by, 'totalNewTradeMoney')
#         data = get_train_std(train, data, by, 'saleSecHouseNum')
        data = get_train_std(train, data, by, 'totalTradeArea')
        data = get_train_std(train, data, by, 'tradeSecNum')
#         data = get_train_std(train, data, by, 'totalNewTradeArea') 
#         data = get_train_std(train, data, by, 'tradeNewNum')
        data = get_train_std(train, data, by, 'remainNewNum')
        data = get_train_std(train, data, by, 'supplyNewNum')
#         data = get_train_std(train, data, by, 'newWorkers')  
        data = get_train_std(train, data, by, 'bedroom')                                
#         data = get_train_std(train, data, by, 'totalWorkers') # 特征选择删除了
#         #中位数
#         data = get_train_median(train, data, by, 'tradeMonth') #加了变差
#         data = get_train_median(train, data, by, 'houseFloor') #加了变差                           
#         data = get_train_median(train, data, by, 'buildYear') #加了变差                              
#         data = get_train_median(train, data, by, 'totalFloor')
#         data = get_train_median(train, data, by, 'pv')
#         data = get_train_median(train, data, by, 'uv')
#         data = get_train_median(train, data, by, 'tradeMeanPrice')
#         data = get_train_median(train, data, by, 'tradeNewMeanPrice')                               
        data = get_train_median(train, data, by, 'totalTradeMoney')
        data = get_train_median(train, data, by, 'totalNewTradeMoney')
#         data = get_train_median(train, data, by, 'saleSecHouseNum') # 特征选择删除了
#         data = get_train_median(train, data, by, 'totalTradeArea')
#         data = get_train_median(train, data, by, 'tradeSecNum')
#         data = get_train_median(train, data, by, 'totalNewTradeArea') 
#         data = get_train_median(train, data, by, 'tradeNewNum')
#         data = get_train_median(train, data, by, 'remainNewNum')
#         data = get_train_median(train, data, by, 'supplyNewNum') # 特征选择删除了
#         data = get_train_median(train, data, by, 'newWorkers')  
#         data = get_train_median(train, data, by, 'bedroom')  # 特征选择删除了
#         data = get_train_median(train, data, by, 'totalWorkers') # 特征选择删除了
        return data                        
####################################################################################                                
    by=['communityName']
    data = groupby(data, by)
    by=['plate']
    data = groupby(data, by)                           
################## 平滑处理 ################################################################                                
    big_num_cols = ['totalTradeMoney','totalTradeArea','tradeMeanPrice','totalNewTradeMoney', 'totalNewTradeArea',
                'tradeNewMeanPrice','remainNewNum', 'supplyNewNum', 'supplyLandArea',
                'tradeLandArea','landTotalPrice','landMeanPrice','totalWorkers','newWorkers',
                'residentPopulation','pv','uv',
                   ]
    num_cols = ['area','subwayStationNum', 'busStationNum', 'interSchoolNum', 'schoolNum',
                'privateSchoolNum', 'hospitalNum', 'drugStoreNum', 'gymNum', 'bankNum', 'shopNum',
                'parkNum', 'mallNum', 'superMarketNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice',
                'tradeSecNum', 'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum',
                'remainNewNum', 'supplyNewNum', 'supplyLandNum', 'supplyLandArea', 'tradeLandNum',
                'tradeLandArea', 'landTotalPrice', 'landMeanPrice', 'totalWorkers', 'newWorkers',
                'residentPopulation', 'pv', 'uv', 'lookNum'] 
    for col in num_cols:
        high = np.percentile(data[col].values, 99.8)
        low = np.percentile(data[col].values, 0.2)
        data.loc[data[col] > high, col] = high
        data.loc[data[col] < low, col] = low
    # 过大量级值取log平滑
    for col in big_num_cols:
        data[col] = data[col].map(lambda x: np.log1p(x))                            

##################################################################################                              
    return data

In [28]:
def w2v(data):
    print(data.shape)
    path = './'
    save_path = path+'w2v'
    sentence = []
    w2v_list = ['communityName']
    for line in list(data[w2v_list].values):
        sentence.append([str(l) for idx, l in enumerate(line)])
    print('training...')
    L=10
    model = Word2Vec(sentence, size=L, window=2, min_count=1, 
                     workers=multiprocessing.cpu_count(),iter=10)
    print('outputing...')
    
    for fea in w2v_list:
        values = []
        for line in list(data[fea].values):
            values.append(line)
        values = set(values)
#         print(fea,len(values))
        # 提取每个词的词向量
        w2v = []
        for i in values:
            a = [i]
            a.extend(model[str(i)])
            w2v.append(a)
        out_df = pd.DataFrame(w2v)
        # 设置列名
        name = [fea]
        for i in range(L):
            name.append(name[0] + 'W' + str(i))
        out_df.columns = name
        out_df.to_csv(save_path + '/' + fea + '.csv', index=False)
    
    
    w2v_path = path+'w2v'
    w2v_features = []
    for col in ['communityName']:
        df = pd.read_csv(w2v_path + '/' + col + '.csv')
        fs = list(df)
        fs.remove(col)
        w2v_features += fs
        data = pd.merge(data, df, on=col, how='left')
    print('word2vec:')
    print(w2v_features)                            
    return data

In [29]:
def remove(data):               
    # 删除特征
    remove_col = [ 
        'totalNewTradeMoney', 
        'help_sum', 
        'tradeNewNum', 
        'supplyLandArea', 
        'totalTradeMoney',
        'landTotalPrice', 
        'uv', 
        'shop_num',
        'shopNum', 
        'schoolNum', 
        'totalNewTradeArea', 
        'region', 
        'landMeanPrice', 
        'totalTradeArea', 
        'superMarketNum', 
        'tradeLandArea',
        'rentType', 
        'gym_bankNum', 
        'communityName', 
        'play_sum',
        'bus_sub_num',
        'area_mean_price',
        'New_area_mean_price',
#         'Mean_area_mean_price',
#         'Mean_price',
                 ]   
    data = data.drop(remove_col,axis=1) 
    return data

In [30]:
# category 处理
def obj2cat(data):
    # 转换object类型数据
#     ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName', 'region', 'plate']
    columns = [ 'rentType','houseFloor', 'houseToward', 'houseDecoration', 'communityName','region', 'plate'] # 
    for col in columns:
        data[col] = data[col].astype('category')
    return data
def cat2obj(data):
    # 转换数据
    columns = [ 'buildYear', 'houseType', 'tradeTime', 'rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName', 'region', 'plate']
    for col in columns:
        data[col] = data[col].astype('object')
    return data

In [31]:
# 数据预处理
def getData(is_test=True,x_train=None,x_val=None,y_train=None):
    ### Load data ###
    if is_test:
        train = pd.read_csv('./train_data.csv')
        test = pd.read_csv('./test_b.csv')
    else:
        train = pd.concat([x_train,y_train],axis=1)
        test = x_val.copy()
        train = cat2obj(train)
        test = cat2obj(test)        
    print('### Before data preprocessing:')
    print("Train: ", train.shape[0], "samples, and ", train.shape[1], "cols")
    print("Test: ", test.shape[0], "samples, and ", test.shape[1], "cols")

    ### Data processing ###
    train = parseData(train,is_test)
    test = parseData(test,is_test)

    train = cleanData(train)

    train = MakeFeatures_1(train)
    test = MakeFeatures_1(test)
                        
    train = w2v(train)
    test = w2v(test)                            
                       
    train = obj2cat(train)
    test = obj2cat(test)
#     train,test = overfit(train,test)
    train = remove(train)
    test = remove(test)
    print('### After data preprocessing:')
    X = train.drop(['tradeMoney'],axis=1)
    y = train['tradeMoney'].copy()
    
    X_test = test.copy()

    print("Train: ", X.shape[0], "samples, and ", X.shape[1], "features")
    print("Test: ", X_test.shape[0], "samples, and ", X_test.shape[1], "features")
    print('Features:', X.columns)
    return X,y,X_test

In [32]:
X_train,Y_train,X_test = getData(is_test=True)

### Before data preprocessing:
Train:  41440 samples, and  51 cols
Test:  2401 samples, and  50 cols
(39616, 129)
training...


d:\tools\anaconda3.5.0\envs\house\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


outputing...


d:\tools\anaconda3.5.0\envs\house\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


word2vec:
['communityNameW0', 'communityNameW1', 'communityNameW2', 'communityNameW3', 'communityNameW4', 'communityNameW5', 'communityNameW6', 'communityNameW7', 'communityNameW8', 'communityNameW9']
(2401, 128)
training...
outputing...
word2vec:
['communityNameW0', 'communityNameW1', 'communityNameW2', 'communityNameW3', 'communityNameW4', 'communityNameW5', 'communityNameW6', 'communityNameW7', 'communityNameW8', 'communityNameW9']
### After data preprocessing:
Train:  39616 samples, and  115 features
Test:  2401 samples, and  115 features
Features: Index(['area', 'houseFloor', 'totalFloor', 'houseToward', 'houseDecoration',
       'plate', 'buildYear', 'saleSecHouseNum', 'subwayStationNum',
       'busStationNum',
       ...
       'communityNameW0', 'communityNameW1', 'communityNameW2',
       'communityNameW3', 'communityNameW4', 'communityNameW5',
       'communityNameW6', 'communityNameW7', 'communityNameW8',
       'communityNameW9'],
      dtype='object', length=115)


In [33]:
#### K-Fold CV #####
# 10-Fold CV
import datetime
kf = KFold(n_splits=5, shuffle=True, random_state=0)
out_of_fold = np.zeros(len(X_train))
predictions = np.zeros(len(X_test))
feature_importance_df = pd.DataFrame()
params = {
    'num_leaves': 31,
    'min_data_in_leaf': 20,
    'min_child_samples':20,
    'objective': 'regression',
    'learning_rate': 0.01,
    "boosting": "gbdt",
    "feature_fraction": 0.8,
    "bagging_freq": 1,
    "bagging_fraction": 0.85,
    "bagging_seed": 23,
    "metric": 'rmse',
    "lambda_l1": 0.2,
    "nthread": 4,
}
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print( "\n[{}] Fold {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), fold+1))
    trn_data = lgb.Dataset(X_train.iloc[train_idx], label=Y_train[train_idx])
    val_data = lgb.Dataset(X_train.iloc[val_idx], label=Y_train[val_idx])
    # Train
    num_round = 10000
    clf = lgb.train(params, trn_data, num_round,valid_sets=[trn_data, val_data], verbose_eval=200,early_stopping_rounds=200)
    out_of_fold[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=clf.best_iteration)

    
    # Predict test data
    predictions += clf.predict(X_test, num_iteration=clf.best_iteration) / kf.n_splits

    # Feature Importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = X_train.columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
#     trn_data = lgb.Dataset(X_train.iloc[train_idx], label=Y_train[train_idx]/X_train.iloc[train_idx]['area'])
#     val_data = lgb.Dataset(X_train.iloc[val_idx], label=Y_train[val_idx]/X_train.iloc[val_idx]['area'])

#     # Train
#     num_round = 10000
#     clf = lgb.train(params, trn_data, num_round,valid_sets=[trn_data, val_data], verbose_eval=200,early_stopping_rounds=200)
#     out_of_fold[val_idx] = (clf.predict(X_train.iloc[val_idx], num_iteration=clf.best_iteration))*X_train.iloc[val_idx]['area']

    
#     # Predict test data
#     predictions += (clf.predict(X_test, num_iteration=clf.best_iteration) / kf.n_splits)*X_test['area']

#     # Feature Importance
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["Feature"] = X_train.columns
#     fold_importance_df["importance"] = clf.feature_importance()
#     fold_importance_df["fold"] = fold + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

print('K-Fold score:{:.6f}'.format(r2_score(out_of_fold, Y_train)))


[2019-08-06 19:20:46] Fold 1


d:\tools\anaconda3.5.0\envs\house\lib\site-packages\lightgbm\basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[200]	training's rmse: 895.868	valid_1's rmse: 914.838
[400]	training's rmse: 726.431	valid_1's rmse: 760.328
[600]	training's rmse: 668.881	valid_1's rmse: 720.066
[800]	training's rmse: 633.471	valid_1's rmse: 698.731
[1000]	training's rmse: 608.662	valid_1's rmse: 685.726
[1200]	training's rmse: 589.079	valid_1's rmse: 676.759
[1400]	training's rmse: 572.796	valid_1's rmse: 670.32
[1600]	training's rmse: 558.78	valid_1's rmse: 665.059
[1800]	training's rmse: 546.463	valid_1's rmse: 661.028
[2000]	training's rmse: 535.596	valid_1's rmse: 658.063
[2200]	training's rmse: 525.785	valid_1's rmse: 655.486
[2400]	training's rmse: 516.762	valid_1's rmse: 653.695
[2600]	training's rmse: 508.402	valid_1's rmse: 652.085
[2800]	training's rmse: 500.615	valid_1's rmse: 650.528
[3000]	training's rmse: 493.191	valid_1's rmse: 649.688
[3200]	training's rmse: 486.286	valid_1's rmse: 648.658
[3400]	training's rmse: 479.653	valid_1's rmse:

Early stopping, best iteration is:
[4739]	training's rmse: 443.405	valid_1's rmse: 652.837
K-Fold score:0.917418


In [34]:
from sklearn.metrics import r2_score
def online_score(pred):
    print("预测结果最大值：{},预测结果最小值：{}".format(pred.max(),pred.min()))
    # a榜测分
    conmbine1 = pd.read_csv("./sub_b_919.csv",engine = "python",header=None)
    score1 = r2_score(pred, conmbine1)
    print("对比919分数:{}".format(score1))

In [35]:
online_score(predictions)

预测结果最大值：15303.416250010467,预测结果最小值：1235.2221321561553
对比919分数:0.9909407147885863


In [ ]:
- 预测结果最大值：15303.416250010467,预测结果最小值：1235.2221321561553
  对比919分数:0.9909407147885863